In [68]:
import langgraph
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import re

load_dotenv()

True

In [69]:
model = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0
)
model.invoke("Good morning")

AIMessage(content='Good morning! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-81848ead-b46e-4bda-9166-a90ec4047dab-0')

In [70]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: BaseMessage
    code_generated: Annotated[Sequence[BaseMessage], operator.add]
    critique_generated: Annotated[Sequence[BaseMessage], operator.add]
    explanation_generated: Annotated[Sequence[BaseMessage], operator.add]
    is_explanation_generated: False
    critique: False
    execute_code: False
    error_occurred: False
    exception_error: Annotated[Sequence[BaseMessage], operator.add]

In [71]:
def coder_node(state):
    last_message = state["messages"]
    last_code = state["code_generated"] 
    last_critique = state["critique_generated"]
    if state["critique"]:
        enhanced_prompt = f"""Here is the code: {last_code}. Here is the critique: {last_critique[-1]}.
        Make modifications to the code based on the critique. ALWAYS ensure to ONLY write code without any additional
        information. The code should ALWAYS start with ```"""
    else:
        enhanced_prompt = f"""Here is the user query: {last_message}. Write a code in
        as much detail as possible. ALWAYS ensure to ONLY write code without any additional
        information. The code should ALWAYS start with ```"""
    response = model.invoke(enhanced_prompt)
    return {"code_generated": [response.content]}

def code_review_node(state):
    messages = state["code_generated"]
    last_code = messages[-1]
    enhanced_prompt = f"""Based on the results from the code, check and 
    critique this code to find out if there are more things to add and there are no mistakes at all: {last_code}"""
    response = model.invoke(enhanced_prompt)
    return {"critique_generated": [response.content], "critique": True}

def executor_node(state):
    if not state["execute_code"]:
        final_code = state["code_generated"][-1]
        code_blocks = re.findall(r"```(python)?(.*?)```", final_code, re.DOTALL)
        code = "\n".join([block[1].strip() for block in code_blocks])
        if code:
            with open("llm_generated_code.py", "w") as file:
                file.write(code)
            try:
                with open("llm_generated_code.py", "r") as file:
                    exec(file.read(), {})
            except Exception as e:
                return {"error_occurred": True, "exception_error": f"{e}"}
        return {"execute_code": True, "error_occurred": False}

def debugger_node(state):
    if state["error_occurred"]:
        messages = state["code_generated"]
        last_code = messages[-1]
        exception_error_generated = state["exception_error"][-1]
        enhanced_prompt = f"""Here is the code: {last_code}. Now 
        this code ran into the following error: {exception_error_generated}.
        Correct the code and try again.
        ALWAYS ensure to ONLY write code without any additional
        information. The code should ALWAYS start with ```
        """
        response = model.invoke(enhanced_prompt)
        return {"code_generated": [response.content]                                                                                     }

def explainer_node(state):
    messages = state["code_generated"]
    last_code = messages[-1]
    enhanced_prompt = f"""Here is the code: {last_code}. 
    Based on the code generated, explain not the code
    but the business impact about what the code is doing.
    It should not be code explanation but explanation in 
    terms of business value it is giving.
    """
    response = model.invoke(enhanced_prompt)
    return {"explanation_generated": [response.content], "is_explanation_generated": True}
        
def should_continue(state):
    if len(state["code_generated"]) < 2:
        return "code_review"
    else:
        return "executor"

def should_continue_executor(state):
    if state["execute_code"] == True and state["error_occurred"] == False:
        return "explainer"
    if state["execute_code"] == True and state["error_occurred"] == True:
        return "debugger"

def should_continue_debugger(state):
    if state["error_occurred"]:
        return "executor"
    else:
        return "explainer"

def should_continue_explainer(state):
    if state["execute_code"] == True and state["error_occurred"] == False:
        return "explainer"
    if state["execute_code"] == True and state["error_occurred"] == True and state["is_explanation_generated"] == True:
        return END

In [72]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("coder", coder_node)
workflow.add_node("code_review", code_review_node)
workflow.add_node("executor", executor_node)
workflow.add_node("debugger", debugger_node)
workflow.add_node("explainer", explainer_node)

workflow.add_conditional_edges("coder", should_continue)
workflow.add_conditional_edges("executor", should_continue_executor)
workflow.add_conditional_edges("debugger", should_continue_debugger)
workflow.add_edge("explainer", END)
workflow.add_edge("code_review", "coder")
workflow.set_entry_point("coder")
# workflow.set_finish_point("explainer")

app = workflow.compile()

In [74]:
input_prompt = """
Build a time series plot of a random dataset and also give explanation 
of the time series. Ensure that the date lies between 2018 to 2023. It 
should be about Computer sales considering all the conditions that actually 
occurred in the world. Use plotly to display the visuals in jupyter notebook
"""

input = {"messages": input_prompt}
response = app.invoke(input)

<string>:6: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

Skipping write for channel None:inbox which has no readers
